In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Extracting the information from the Disney Plus file
datafile1 = "Resources/disney_plus_shows.csv"
datafile2 = "Resources/IMDb.csv"
datafile3 = "Resources/Platforms.csv"

In [3]:
# Turning first csv into a dataframe
disney_df = pd.read_csv(datafile1)

In [4]:
# Droping unrequired columns 
disney_cleaned = disney_df[["title","director","country","year","genre","imdb_rating"]]
disney_cleaned = disney_cleaned.rename(columns={"imdb_rating":"rating"})
disney_cleaned ['year'] = disney_cleaned ['year'].str.split('–', n = 0, expand = True)

In [5]:
# Turning second csv into a dataframe
imdb_df= pd.read_csv(datafile2)

c:\users\jonat\desktop\bootcamp\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Droping unrequired columns 
imdb_cleaned = imdb_df[["title","director","country","year","genre","avg_vote"]]
imdb_cleaned = imdb_cleaned.rename(columns={"avg_vote":"rating"})

In [7]:
# Turning third csv into a dataframe
platforms_df= pd.read_csv(datafile3)

In [8]:
# Droping unrequired columns 
platforms_cleaned = platforms_df[["Title","Directors","Country","Year","Genres","IMDb"]]
platforms_cleaned=platforms_cleaned.rename(columns={"Title":"title","Directors":"director",
                                                    "Country":"country",
                                                    "Year":"year",
                                                    "Genres":"genre",
                                                    "IMDb":"rating"})

In [9]:
# Append three df's to create a DB
consolidated_df = disney_cleaned.append([imdb_cleaned, platforms_cleaned])

In [10]:
# Clean DB's headers and remove duplicates
movie_sn = consolidated_df.loc[:, ["title", "director", "country", "year", "genre","rating"]]
movie_sn = movie_sn.drop_duplicates()
movie_sn.dropna(inplace=True)
movie_sn.head(5)

,title,director,country,year,genre,rating
0,10 Things I Hate About You,Gil Junger,USA,1999,"Comedy, Drama, Romance",7.3
2,101 Dalmatians,Stephen Herek,"USA, UK",1996,"Adventure, Comedy, Crime, Family",5.7
3,101 Dalmatians 2: Patch's London Adventure,"Jim Kammerud, Brian Smith",USA,2002,"Animation, Adventure, Comedy, Family, Musical",5.8
4,102 Dalmatians,Kevin Lima,"USA, UK",2000,"Adventure, Comedy, Family",4.9
5,12 Dates of Christmas,James Hayman,USA,2011,"Comedy, Fantasy, Romance",6.3


In [11]:
# Create Directors, Country, Release Year and IMDb score Tables
director_summary = movie_sn[["director", "title"]]
country_summary = movie_sn[["country", "title"]]
release_summary = movie_sn[["year", "title"]]
imdb_summary = movie_sn[["rating", "title"]]

In [12]:
# Create connection and engine
rds_connection_string = "postgres:Star18Dust@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
# Export to SQL and CSV

# director summary
director_summary.to_sql(name='director_summary', con=engine, if_exists='append', index=False)
director_summary.to_csv("director_summary", index=True)

# country summary
country_summary.to_sql(name='country_summary', con=engine, if_exists='append', index=False)
country_summary.to_csv("country_summary", index=True)

# release summary
release_summary.to_sql(name='release_summary', con=engine, if_exists='append', index=False)
release_summary.to_csv("release_summary", index=True)

# imdb summary
imdb_summary.to_sql(name='imdb_summary', con=engine, if_exists='append', index=False)
imdb_summary.to_csv("imdb_summary", index=True)

In [16]:
engine.table_names()

[]